<a href="https://colab.research.google.com/github/alhichri/BlindSys/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_applications

In [ ]:
from os import listdir
import numpy as np
import itertools
import scipy.io
import matplotlib.pyplot as plt
from pickle import load, dump
import h5py
import time

from keras import backend as K
K.set_image_data_format('channels_last')
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
#from keras.applications.vgg16 import preprocess_input
from keras.applications import inception_v3
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.utils import to_categorical
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Activation, BatchNormalization, Conv1D,GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import ReLU, LeakyReLU, PReLU, ELU, Conv2D, MaxPooling2D, Flatten
from keras.layers.merge import concatenate, add, maximum
from keras.preprocessing import image
from keras.utils import plot_model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, classification_report
from sklearn.metrics import hamming_loss, label_ranking_loss, zero_one_loss
from sklearn.utils.multiclass import unique_labels

from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

In [3]:
from importlib.machinery import SourceFileLoader
somemodule = SourceFileLoader('squeezenet', '/content/gdrive/My Drive/000papers/utils/squeezenet.py').load_module()
from squeezenet import SqueezeNet
#DNet1 = SqueezeNet(input_shape=img_dim, include_top=True,weights='imagenet')

In [4]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def plot_confusion_matrix2(cm, title='Confusion matrix', cmap=plt.cm.Oranges):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(cm.shape[1])
    plt.xticks(tick_marks, rotation=45)
    ax = plt.gca()
    ax.set_xticklabels((ax.get_xticks() +1).astype(str))
    plt.yticks(tick_marks)
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +  ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(20)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",  fontsize=28, rotation=45)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



def Evaluate(ACTUAL,PREDICTED):
#% This fucntion evaluates the performance of a classification model by 
#% calculating the common performance measures: Accuracy, Sensitivity, 
#% Specificity, Precision, Recall, F-Measure, G-mean.
#% Input: ACTUAL = Column matrix with actual class labels of the training
#%                 examples
#%        PREDICTED = Column matrix with predicted class labels by the
#%                    classification model
#% Output: EVAL = Row matrix with all the performance measures

    idx = ACTUAL==1
    p = len(ACTUAL[idx])
    n = len(ACTUAL[~idx])
    N = p+n;
    tp = np.sum(ACTUAL[idx]==PREDICTED[idx])
    tn = np.sum(ACTUAL[~idx]==PREDICTED[~idx])
    fp = n-tn
    fn = p-tp
    tp_rate = tp/p
    tn_rate = tn/n
    
    accuracy = (tp+tn)/N
    sensitivity = tp_rate
    specificity = tn_rate
    precision = tp/(tp+fp)
    recall = sensitivity
    f_measure = 2*((precision*recall)/(precision + recall))
    gmean = np.sqrt(tp_rate*tn_rate)
    HL = hamming_loss(ACTUAL,PREDICTED)
    ZeroOneLoss = zero_one_loss(ACTUAL,PREDICTED)
    #print('ZeroOneLoss: ', ZeroOneLoss)

    #disp('      accuracy  | sensitivity | specificity | precision |  recall  |  f_measure  |   gmean')
    EVAL = np.empty(9,)
    EVAL[0] = accuracy; EVAL[1] = sensitivity; EVAL[2] = specificity;
    EVAL[3] = precision; EVAL[4] = recall; EVAL[5] = f_measure; EVAL[6] = gmean; 
    EVAL[7] = HL;  EVAL[8] = ZeroOneLoss
    EVAL = 100*EVAL

    return EVAL


def Evaluate_Multilabel_classification(ACTUAL,PREDICTED, number_of_labels):
    Complete_Evaluation = np.zeros((number_of_labels , 9))
    Complete_Evaluation[0,:] = Evaluate(ACTUAL[:,0], PREDICTED[:,0])
    Summary_Evaluation = (1/number_of_labels)*Complete_Evaluation[0,:]
    for i in range(1,number_of_labels):
        Complete_Evaluation[i,:] = Evaluate(ACTUAL[:,i], PREDICTED[:,i])
        Summary_Evaluation = Summary_Evaluation + (1/number_of_labels)*Complete_Evaluation[i,:]
    RankingLoss = label_ranking_loss(ACTUAL,PREDICTED)


    return Complete_Evaluation, Summary_Evaluation

def trainModel(VGG16Model,Xtrain, ytrain, learningRate=0.001,epochs=100, batch_size=32):
        VGG16Model.compile(loss=['mse'], optimizer=Adam(lr= learningRate ), metrics=['mse','mae'])
        Loss = []
        for epoch in range(epochs):
            t1 = time.time()
            if (epoch+1) % 100 == 0:
                learningRate = learningRate * 0.1
                VGG16Model.compile(loss=['mse'], optimizer=Adam(lr=learningRate),metrics=['mse','mae'])
            print('Epoch {} of {}'.format(epoch + 1, epochs))
            n_batches = num_train_samples / batch_size
            # print(n_batches)
            idx_train_shuffle_1 = np.arange(Xtrain.shape[0])
            np.random.shuffle(idx_train_shuffle_1)
            epoch_loss = []
            # epoch_t2_loss = []
            index = 0
            while index < n_batches :
                t1_data = Xtrain[idx_train_shuffle_1[index * batch_size:(index + 1) * batch_size]]
                t1_label = ytrain[idx_train_shuffle_1[index * batch_size:(index + 1) * batch_size]]
                if t1_label.shape[0] < 1:
                    idx = np.random.randint(0, Xtrain.shape[0], batch_size)
                    t1_data = Xtrain[idx]
                    t1_label = ytrain[idx]
                Loss1 = VGG16Model.train_on_batch(t1_data, t1_label)
                epoch_loss.append(Loss1[0])
                index += 1
            print('\n[Loss : {:.3f} ] Time'.format(np.mean(epoch_loss)), time.time() - t1)
            Loss.append(np.mean(epoch_loss))
        print('\n[Loss : {:.3f} ] Time'.format(np.mean(epoch_loss)), time.time() - t0)
        return VGG16Model, np.array(Loss)


def FitModel(The_model, file_name, Xtrain, ytrain, learningRate, epochs, batch_size ):

    The_model.compile(loss=['mse'], optimizer=Adam(lr=0.001), metrics=['mse','mae'])
    model_history0 = The_model.fit(Xtrain, ytrain, 
                                batch_size=batch_size, 
                                epochs=80, 
                                validation_split=0.2,
                                verbose=1, shuffle=True)    
    The_model.compile(loss=['mse'], optimizer=Adam(lr=0.0001), metrics=['mse','mae'])
    
    mc = ModelCheckpoint(file_name, monitor='mse', 
                         mode='min', verbose=1, 
                         save_best_only=True)
    # patient early stopping
    es=EarlyStopping(monitor='loss', min_delta=0.00005, patience=20, \
                          verbose=1, mode='auto')
    model_history1 = The_model.fit(Xtrain, ytrain, 
                                batch_size=batch_size, 
                                epochs=40, 
                                verbose=1, shuffle=True,
                                callbacks=[es,mc])                                
    model_history0.history['loss'] = model_history0.history['loss'] +  model_history1.history['loss']  
    #model_history0.history['acc'] = model_history0.history['acc'] +  model_history1.history['acc']  


    

In [ ]:
dataset = 0
if dataset==0:
    path = '/content/gdrive/My Drive/Multi_objects_datasets/'
    mat = scipy.io.loadmat(path+'ksu1.mat')
    X = mat['trainall'][0];  Y = mat['testall'][0]
    mat = scipy.io.loadmat(path+'ksu1Labels.mat')
    L_train = mat['Labels_training']; L_test = mat['Labels_test'];  Names = mat['names']
    num_samples = len(X)
    object_set = ['Pillar', 'Fire extinguisher/hose', 'Trash can', 'Chairs', 'External Door', 'Hallway', 'Self-service', 'Reception', 'Didactic service machine', 'Display Screen', 'Board', 'Stairs', 'Elevator', 'Laboratory', 'Internal Door']
elif dataset == 1:
    path = '/content/gdrive/My Drive/Multi_objects_datasets/'
    mat = scipy.io.loadmat(path+'ksu2.mat')
    X = mat['trainall'][0];  Y = mat['testall'][0]
    mat = scipy.io.loadmat(path+'ksu2Labels.mat')
    L_train = mat['Labels_training']; L_test = mat['Labels_test'];  Names = mat['names']
    num_samples = len(X)
    object_set = ['Board', 'Fire extinguisher/hose', 'Trash cans', 'Chairs', 'External door', 'didactic service machine', 'Self-service', 'Reception', 'Cafeteria', 'Display screen', 'Pillar', 'Stairs', 'Elevator', 'Prayer room', 'Internal door']
elif dataset == 2:
    path = '/content/gdrive/My Drive/Multi_objects_datasets/'
    mat = scipy.io.loadmat(path+'UTrento1.mat')
    X = mat['trainall'][0];  Y = mat['testall'][0]
    mat = scipy.io.loadmat(path+'UTrento1Labels.mat')
    L_train = mat['Labels_training']; L_test = mat['Labels_test'];  Names = mat['names']
    num_samples = len(X)
    object_set = ['External Window', 'Board', 'Table', 'External Door', 'Stair Door', 'Access Control Reader', 'Ofﬁce', 'Pillar', 'Display Screen', 'People', 'ATM', 'Chairs', 'Bins', 'Internal Door',  'Elevator']
elif dataset==3:
    path = '/content/gdrive/My Drive/Multi_objects_datasets/'
    mat = scipy.io.loadmat(path+'UTrento2.mat')
    X = mat['trainall'][0];  Y = mat['testall'][0]
    mat = scipy.io.loadmat(path+'UTrento2Labels.mat')
    L_train = mat['Labels_training']; L_test = mat['Labels_test'];  Names = mat['names']
    num_samples = len(X)
    object_set = ['Stairs', 'Heater', 'Corridor', 'Board', 'Laboratories', 'Bins', 'Ofﬁce', 'People', 'Pillar', 'Elevator', 'Reception', 'Chairs', 'Self-Service', 'External Door', 'Display Screen']    

Xtrain = np.stack(X)
Xtest = np.stack(Y)

# process inputs, removing mean
# must cionvert each image to arrauy first : test_img = np.array(test_img)
#X = preprocess_input(np.float6a4(X))
#X = X/255.0
X = preprocess_input(np.float64(Xtrain))
Xtest = preprocess_input(np.float64(Xtest))

ytrain = L_train.T; ytrain = ytrain.astype(int)
ytest = L_test.T; ytest = ytest.astype(int)

# make the number of traning samples a multiple of 32
num_classes = len(Names)
num_train_samples = len(Xtrain)
temp = int(np.floor(num_train_samples/32)+1)
new_train_size = temp*32
extra_samples = new_train_size - num_train_samples
Xtrain = np.vstack((X, X[:extra_samples] ))
ytrain = np.vstack((ytrain , ytrain[:extra_samples]))

num_train_samples = len(Xtrain)

img_rows, img_cols, img_channels = Xtrain[0].shape[0], Xtrain[0].shape[1], Xtrain[0].shape[2]
# Parameters for the DenseNet model builder
(img_channels, img_rows, img_cols) if K.image_data_format() == 'channels_first' else (
img_rows, img_cols, img_channels)

print('Xtrain shape: ', Xtrain.shape)
print('ytrain shape: ', ytrain.shape)
print('Xtest shape: ', Xtest.shape)
print('ytest shape: ', ytest.shape)

frequency_of_objects = np.sum(ytrain, axis = 0)
for i in range(num_classes):
    print('Object: ', i, ' number of occurences: ' , frequency_of_objects[i]  )




In [ ]:

range_0_to_14 = np.expand_dims (  np.arange(0,15,1).T  , axis=1)
batch_size = 32 #int(np.floor(num_train_samples/4)+2) #np.min([100 , 0.4*num_train_samples]).astype(int)      
k = 0;  dataset_OA = np.zeros((1,10)) ; times_arr = [] 
while k<1:  

################################ squeezeNet model   ################################
    img_dim = [img_rows,img_cols,img_channels]
    DNet1 = SqueezeNet(input_shape=img_dim, include_top=True,weights='imagenet')
    #DNet1.summary()
    Net_layers=len(DNet1.layers)
    DNet1.trainable=False
    layer_name = 'fire9/concat'
    x = DNet1.get_layer(layer_name).output
    x = Conv2D(filters=256 , kernel_size=(1,1) , strides=(1,1))(x)
    x = LeakyReLU(alpha=0.5)(x)
    x = BatchNormalization()(x)
    last_layer = GlobalAveragePooling2D()(x)
    classifier2 = Dense(num_classes, activation='linear')(last_layer)
    SQNetModel = Model(inputs=DNet1.inputs, outputs=classifier2)
    SQNetModel.summary()
    #raise ValueError("Script End")
    
    DNet2 = SqueezeNet(input_shape=img_dim, include_top=True,weights='imagenet')
    #DNet1.summary()
    Net_layers=len(DNet2.layers)
    DNet2.trainable=False
    layer_name = 'fire9/concat'
    x = DNet2.get_layer(layer_name).output
    x = Conv2D(filters=256 , kernel_size=(1,1) , strides=(1,1))(x)
    x = LeakyReLU(alpha=0.5)(x)
    x = BatchNormalization()(x)
    last_layer = GlobalAveragePooling2D()(x)
    classifier2 = Dense(num_classes, activation='linear')(last_layer)
    SQNetModel4error = Model(inputs=DNet2.inputs, outputs=classifier2)
    SQNetModel4error.summary()

    t0 = time.time();                  
    file_name = 'SQNetModel_best.h5'
    SQNetModel,model_history1_0 = trainModel(SQNetModel , Xtrain, ytrain, learningRate=0.001 , epochs=2, batch_size=32 )
    plt.plot(model_history1_0)
#            SQNetModel,model_history1_0 = FitModel(SQNetModel, file_name , Xtrain, ytrain, learningRate=0.001 , epochs=60, batch_size=32 )
#            plot_history(model_history1_0) 



    Xtrain_outputs1 = SQNetModel.predict(Xtrain, batch_size=32)
    Xtrain_errors1 = Xtrain_outputs1 - ytrain
    file_name = 'SQNetModel4error_best.h5'
    SQNetModel4error,model_history1_1 = trainModel(SQNetModel4error , Xtrain, Xtrain_errors1, learningRate=0.001 , epochs=2, batch_size=32 )
    plt.plot(model_history1_1)
#            SQNetModel4error,model_history1_1 = FitModel(SQNetModel, file_name , Xtrain, errorInPrediction, learningRate=0.001 , epochs=30, batch_size=32 )
#            plot_history(model_history1_1)           
    time_for_model1 = time.time() -t0

    Prob_task1 = SQNetModel.predict(Xtest, batch_size=32)           
    errorPrediction1 = SQNetModel4error.predict(Xtest, batch_size=32)
    mseErrorPredictionSqueezeNet = mean_squared_error(errorPrediction1,Prob_task1-ytest)


################################ VGG16 model   ################################
    img_dim = (img_rows,img_cols,img_channels)
    DNet3=VGG16(input_shape=img_dim,include_top=False, weights='imagenet', classes=1000)
    #DNet1.summary()
    Net_layers=len(DNet3.layers)
    DNet3.trainable=False
    for layer in range(Net_layers-2):
          DNet3.layers[layer].trainable = False

    x = DNet3.output  # check which one is better??
    #x = BatchNormalization()(feature_tensor)
    feature_tensor = GlobalAveragePooling2D(name='branch1_02')(x)
    x = Dense(128)(feature_tensor)
    x = BatchNormalization()(x)
    last_layer = LeakyReLU(0.5)(x)
    classifier2 = Dense(num_classes, activation='linear')(last_layer)


    VGG16Model = Model(inputs=DNet3.inputs, outputs=classifier2)
    VGG16Model.summary()

    img_dim = (img_rows,img_cols,img_channels)
    DNet4=VGG16(input_shape=img_dim,include_top=False, weights='imagenet', classes=1000)
    #DNet1.summary()
    Net_layers=len(DNet4.layers)
    DNet1.trainable=False
    for layer in range(Net_layers-2):
          DNet4.layers[layer].trainable = False
    x = DNet4.output  # check which one is better??
    #x = BatchNormalization()(feature_tensor)
    feature_tensor = GlobalAveragePooling2D(name='branch1_02')(x)
    x = Dense(128)(feature_tensor)
    x = BatchNormalization()(x)
    last_layer = LeakyReLU(0.5)(x)
    classifier2 = Dense(num_classes, activation='linear')(last_layer)
    VGG16Model4error = Model(inputs=DNet4.inputs, outputs=classifier2)
    VGG16Model4error.summary()

    t0 = time.time();    batch_size = 16               
    VGG16Model,model_history2_0 = trainModel(VGG16Model , Xtrain, ytrain, learningRate=0.001 , epochs=2, batch_size=16 )
    plt.plot(model_history2_0)      

    Xtrain_outputs2 = VGG16Model.predict(Xtrain, batch_size=32)
    Xtrain_errors2 = Xtrain_outputs2 - ytrain
    VGG16Model4error,model_history2_1 = trainModel(VGG16Model4error , Xtrain, Xtrain_errors2, learningRate=0.001 , epochs=2, batch_size=16 )
    plt.plot(model_history2_1)

    time_for_model2 = time.time() -t0


    #This works but I don't know what to do with it:
    # multilabel_confusion_matrix(ytest,Test_predicted)
    # Carry out classification
    t3 = time.time()
    Prob_task1 = SQNetModel.predict(Xtest, batch_size=32)           
    errorPrediction1 = SQNetModel4error.predict(Xtest, batch_size=32)
    mseErrorPredictionSqueezeNet = mean_squared_error(errorPrediction1,Prob_task1-ytest)
    del SQNetModel, SQNetModel4error
    Prob_task2 = VGG16Model.predict(Xtest, batch_size=32)
    errorPrediction2 = VGG16Model4error.predict(Xtest, batch_size=32)
    mseErrorPredictionVGG16 = mean_squared_error(errorPrediction2,Prob_task2-ytest)

    time_per_image = (time.time() - t3) / len(Xtest)
    times_arr = np.hstack((times_arr , time_per_image))

    #del VGG16Model, VGG16Model4error

    ###################  Save current results ##############################

    # runNumber = dataset; path = ''
    # fname = path + 'predictionresults%i.npz' % runNumber
    # D = {'Prob_task1': Prob_task1, 'Prob_task2': Prob_task2 , 
    #       'errorPrediction1': errorPrediction1, 'errorPrediction2': errorPrediction2}
    # np.savez( fname , **D)
    # print('Saving prediction results in file called predictionresults%i.npz' % runNumber)


    ###################  load and fuse ##############################
    ###################  load and fuse ##############################

    # runNumber = 1; path=''
    # fname = path + 'predictionresults%i.npz' % runNumber
    # D = np.load(fname)
    # Prob_task1 = D['Prob_task1']; 
    # Prob_task2 = D['Prob_task2']; 
    # errorPrediction1 = D['errorPrediction1']; 
    # errorPrediction2 = D['errorPrediction2']; 


    ###################  Fuse Results using OWA ##############################

    Prob_task = Prob_task1.copy()
    for obj in range(15):
        #print(obj)
        e1 = 1 /(1+np.abs( errorPrediction1[:,obj] ) )
        e2 = 1 / (1+np.abs( errorPrediction2[:,obj]  )  )
        w1 = e1.copy(); w2 = e2.copy()
        for sample in range(len(e1)):
            if e2[sample]>e1[sample]:
                w2[sample] = 1/(1+e2[sample])
                w1[sample] =  e2[sample] / (1+e2[sample])
            else:
                w1[sample] = 1/(1+e1[sample])
                w2[sample] =  e1[sample] / (1+e1[sample])   
#                w1 = 1 - e1 / (e1+e2)
#                w2 = 1 - e2 / (e1+e2)
        Prob_task[:,obj] = w1 * Prob_task1[:,obj] + w2* Prob_task2[:,obj]  


    bestThresh = 0.3
    AVG1 = np.zeros((  8 ))
    print('accuracy  | sensitivity | specificity | precision |  recall  |  f_measure  |   gmean')
    for i in (1,2,3,4,5,6,7,8):
        Test_predicted1 = Prob_task1 > i*0.1;   Test_predicted1 =Test_predicted1 *1
        Complete_Evaluation1, Summary_Evaluation1 = Evaluate_Multilabel_classification(ytest,Test_predicted1, num_classes)
        AVG1[i-1] = ( (Summary_Evaluation1[1] + Summary_Evaluation1[2])/2 )
    Test_predicted1 = Prob_task1 > bestThresh;   Test_predicted1 =Test_predicted1 *1
    Complete_Evaluation1, Summary_Evaluation1 = Evaluate_Multilabel_classification(ytest,Test_predicted1, num_classes)
    dataset_OA = np.vstack((dataset_OA , np.hstack(((Summary_Evaluation1[1] + Summary_Evaluation1[2])/2 , Summary_Evaluation1))   ))

    AVG2 = np.zeros((  8 ))
    for i in (1,2,3,4,5,6,7,8):
        Test_predicted2 = Prob_task2 > i*0.1;   Test_predicted2 =Test_predicted2 *1
        Complete_Evaluation2, Summary_Evaluation2 = Evaluate_Multilabel_classification(ytest,Test_predicted2, num_classes)
        AVG2[i-1] = ( (Summary_Evaluation2[1] + Summary_Evaluation2[2])/2 )
    Test_predicted2 = Prob_task2 > bestThresh;   Test_predicted2 =Test_predicted2 *1
    Complete_Evaluation2, Summary_Evaluation2 = Evaluate_Multilabel_classification(ytest,Test_predicted2, num_classes)
    dataset_OA = np.vstack((dataset_OA , np.hstack(((Summary_Evaluation2[1] + Summary_Evaluation2[2])/2 , Summary_Evaluation2))   ))

    AVG3 = np.zeros((  8 ))
    for i in (1,2,3,4,5,6,7,8):
        Test_predicted3 = Prob_task > i*0.1;   Test_predicted3 =Test_predicted3 *1
        Complete_Evaluation3, Summary_Evaluation3 = Evaluate_Multilabel_classification(ytest,Test_predicted3, num_classes)
        AVG3[i-1] = ( (Summary_Evaluation3[1] + Summary_Evaluation3[2])/2 )
    Test_predicted3 = Prob_task > bestThresh;   Test_predicted3 =Test_predicted3 *1
    Complete_Evaluation3, Summary_Evaluation3 = Evaluate_Multilabel_classification(ytest,Test_predicted3, num_classes)
    dataset_OA = np.vstack((dataset_OA , np.hstack(((Summary_Evaluation3[1] + Summary_Evaluation3[2])/2 , Summary_Evaluation3))   ))

    k = k +1

In [ ]:
fig, ax = plt.subplots()
ax.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8], AVG1, '--b', label='SqueezeNet')
ax.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8], AVG2, '-r', label='VGG16')
ax.plot([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8], AVG3, '-ok', label='Fused')
#ax.axis('equal')
plt.xlabel('Threshold'); plt.ylabel('AVG(SEN+SPE)')
leg = ax.legend();
ax.legend(loc='upper right', frameon=False)